In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型园 - 稳定扩散V1.5（Dreambooth+LoRA微调）

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_finetuning_dreambooth_lora.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_finetuning_dreambooth_lora.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_finetuning_dreambooth_lora.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在Vertex AI工作台中打开
    </a>
    (推荐使用Python-3 CPU笔记本)
  </td>
</table>

## 概述

本笔记本演示如何在[Colab](https://colab.research.google.com)或[Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench)上使用[Dreambooth和LoRA](https://huggingface.co/docs/diffusers/training/dreambooth#finetuning-with-lora)对[runwayml/stable-diffusion-v1-5](https://huggingface.co/runwayml/stable-diffusion-v1-5)模型进行微调，并在Vertex AI上部署以进行在线预测。


### 目标

- 使用[Dreambooth + LoRA](https://huggingface.co/docs/diffusers/training/dreambooth#finetuning-with-lora)对stable-diffusion-v1.5模型进行微调。
- 将微调后的模型上传到[Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)。
- 将微调后的模型部署到[Vertex AI Endpoint资源](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints)。
- 为文本到图像的在线预测运行。

### 费用

本教程使用Google Cloud的可计费组件：

* Vertex AI
* 云存储

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成费用估算。

## 在开始之前

**注意**：Jupyter会将以`!`为前缀的行作为shell命令运行，并将以`$`为前缀的Python变量插入到这些命令中。

设置笔记本

只在Colab上运行以下命令。如果您正在使用Workbench，请跳过此部分。

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()
    # Install gdown for downloading example training images.
    ! pip3 install gdown
    # Remove wrong cublas version.
    ! pip3 uninstall nvidia_cublas_cu11 --yes

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

仅工作台
1. 点击[此链接](https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_deployment_1_5.ipynb)将笔记本部署到Vertex AI工作台实例。
2. 选择“创建新笔记本”。
3. 点击“高级选项”。
4. 在“环境”选项卡中，选择“Debian 10”作为“操作系统”，并选择“自定义容器”作为“环境”。
5. 将“Docker容器镜像”设置为“us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/transformers-notebook”。
6. 在“机器配置”下，选择1个“T4” GPU，并选择“自动为我安装NVIDIA GPU驱动程序”。
7. 点击“创建”以创建Vertex AI工作台实例。

### 设置Google Cloud项目

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。在您第一次创建帐户时，您将获得300美元的免费信用额用于计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

4. [创建一个云存储存储桶](https://cloud.google.com/storage/docs/creating-buckets) 用于存储实验输出。

5. [创建一个服务帐号](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) 并为其分配 `Vertex AI用户` 和 `存储对象管理员` 角色，以将优化后的模型部署到Vertex AI端点。

为实验环境填写以下变量：

In [1]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = ""  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = ""  # @param {type:"string"}

# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

初始化Vertex-AI API:

In [2]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

### 定义常数

In [3]:
# The pre-built training docker images. They contain training scripts and models.
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-peft-train:20240202_1425_RC00"

# The pre-built serving docker images. They contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-diffusers-serve-opt:20240605_1400_RC00"

### 定义常见的函数

In [8]:
import base64
import glob
import os
from datetime import datetime
from io import BytesIO

import requests
from google.cloud import aiplatform, storage
from PIL import Image


def create_job_name(prefix):
    user = os.environ.get("USER")
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    job_name = f"{prefix}-{user}-{now}"
    return job_name


def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


def image_to_base64(image, format="JPEG"):
    buffer = BytesIO()
    image.save(buffer, format=format)
    image_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return image_str


def base64_to_image(image_str):
    image = Image.open(BytesIO(base64.b64decode(image_str)))
    return image


def image_grid(imgs, rows=2, cols=2):
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid


def deploy_model(
    model_name,
    base_model_id,
    lora_id,
    task,
    machine_type="g2-standard-8",
    accelerator_type="NVIDIA_L4",
):
    """Deploys trained models into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "MODEL_ID": base_model_id,
        "TASK": task,
        "DEPLOY_SOURCE": "notebook",
    }
    if lora_id:
        serving_env["LORA_ID"] = lora_id
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/diffusers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=1,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
    )
    return model, endpoint


def get_bucket_and_blob_name(filepath):
    # The gcs path is of the form gs://<bucket-name>/<blob-name>
    gs_suffix = filepath.split("gs://", 1)[1]
    return tuple(gs_suffix.split("/", 1))


def upload_local_dir_to_gcs(local_dir_path, gcs_dir_path):
    """Uploads files in a local directory to a GCS directory."""
    client = storage.Client()
    bucket_name = gcs_dir_path.split("/")[2]
    bucket = client.get_bucket(bucket_name)
    for local_file in glob.glob(local_dir_path + "/**"):
        if not os.path.isfile(local_file):
            continue
        filename = local_file[1 + len(local_dir_path) :]
        gcs_file_path = os.path.join(gcs_dir_path, filename)
        _, blob_name = get_bucket_and_blob_name(gcs_file_path)
        blob = bucket.blob(blob_name)
        blob.upload_from_filename(local_file)
        print("Copied {} to {}.".format(local_file, gcs_file_path))

用Dreambooth和LoRA进行微调

### 微调

本节使用[dreambooth](https://dreambooth.github.io/)和[LoRA](https://arxiv.org/abs/2106.09685)来微调[stable-diffusion-v1.5](https://huggingface.co/runwayml/stable-diffusion-v1-5)模型，使用[5张狗的图片](https://drive.google.com/drive/folders/1BO_dyz-p65qhBRRMRA4TbZ8qW4rB99JZ)来个性化文本生成图像的模型。

在微调作业完成后，完整的模型将被保存，可以通过[StableDiffusionPipeline](https://huggingface.co/docs/diffusers/api/pipelines/stable_diffusion/text2img)加载以进行推理。

注：在使用LoRA时，可以使用比普通dreambooth更高的学习率。这里我们使用`1e-4`，而不是通常的`2e-6`。

In [ ]:
# Download example training images.
!gdown --folder https://drive.google.com/drive/folders/1BO_dyz-p65qhBRRMRA4TbZ8qW4rB99JZ

# Upload data to Cloud Storage bucket.
upload_local_dir_to_gcs("dog", f"gs://{GCS_BUCKET}/dreambooth-lora/dog")
upload_local_dir_to_gcs("dog", f"gs://{GCS_BUCKET}/dreambooth-lora/dog_class")

注意：如果由于权限不足导致上传步骤失败，您需要为笔记本的云账户[授予存储对象管理员角色](https://cloud.google.com/storage/docs/access-control/using-iam-permissions)。

In [ ]:
# The pre-trained model to be loaded.
model_id = "runwayml/stable-diffusion-v1-5"

# Input and output path.
instance_dir = f"/gcs/{GCS_BUCKET}/dreambooth-lora/dog"
output_dir_dreambooth_lora = f"/gcs/{GCS_BUCKET}/dreambooth-lora/output"
gs_output_dir_dreambooth_lora = f"gs://{GCS_BUCKET}/dreambooth-lora/output"


# Worker pool spec.
machine_type = "n1-highmem-8"
num_nodes = 1
gpu_type = "NVIDIA_TESLA_V100"
num_gpus = 1

# Setup training job.
job_name = create_job_name("dreambooth-lora-stable-diffusion")
job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name,
    container_uri=TRAIN_DOCKER_URI,
)

# Set task to "text-to-image-dreambooth-lora" to finetune using Dreambooth and
# LoRA.
# Pass training arguments and launch job.
# See https://github.com/huggingface/diffusers/blob/87ae330056f6942817656c8f7146283e90cf986b/examples/dreambooth/train_dreambooth_lora.py#L142
# for a full list of training arguments.
model = job.run(
    args=[
        "--task=text-to-image-dreambooth-lora",
        f"--pretrained_model_name_or_path={model_id}",
        f"--instance_data_dir={instance_dir}",
        f"--output_dir={output_dir_dreambooth_lora}",
        "--instance_prompt='a photo of sks dog'",
        "--resolution=512",
        "--train_batch_size=1",
        "--gradient_accumulation_steps=1",
        "--checkpointing_steps=100",
        "--learning_rate=1e-2",
        "--lr_scheduler=constant",
        "--lr_warmup_steps=0",
        "--max_train_steps=50",
        "--validation_prompt='A photo of sks dog in a bucket'",
        "--validation_epochs=10",
        "--seed=0",
    ],
    replica_count=num_nodes,
    machine_type=machine_type,
    accelerator_type=gpu_type,
    accelerator_count=num_gpus,
)

### 部署

为文本到图像任务部署稳定的扩散模型。

部署完成后，您可以将一批文本提示发送到端点以生成图像。

In [ ]:
model, endpoint = deploy_model(
    model_name=create_job_name(prefix="lora-stable-diffusion"),
    model_id=model_id,
    lora_id=gs_output_dir_dreambooth_lora,
    task="text-to-image",
)

请注意：在部署成功后，将下载模型权重。

---

因此，在上述模型部署步骤成功后，需要额外等待5分钟，然后再运行下面的下一步。否则，当您向端点发送请求时，可能会出现“ServiceUnavailable: 503 502:Bad Gateway”错误。

In [ ]:
instances = [
    {"prompt": "A picture of a sks dog in a house"},
    {"prompt": "A picture of a sks dog catching a frisbee"},
    {"prompt": "A picture of a sks dog in front of a computer"},
    {"prompt": "A picture of a sks dog in a bucket"},
]
response = endpoint.predict(instances=instances)
images = [base64_to_image(image) for image in response.predictions]
image_grid(images)

清理

In [ ]:
# Undeploy model and delete endpoints.
endpoint.delete(force=True)

# Delete models.
model.delete()